# Builder123D Sandbox

## Important Links:
* [Builder123D Github](https://github.com/gumyr/build123d)
    * [Documentation](https://build123d.readthedocs.io/en/latest)
* [VSCode Visualization Plugin](https://marketplace.visualstudio.com/items?itemName=bernhard-42.ocp-cad-viewer)
    * [Source](https://github.com/bernhard-42/vscode-ocp-cad-viewer/tree/main)
    * [3CAD Viewer](https://github.com/bernhard-42/three-cad-viewer?tab=readme-ov-file), base for the plugin.
    * [Animation Primer](https://github.com/bernhard-42/bd_animation)
* [Simple CAM package](https://ocp-freecad-cam.readthedocs.io/en/latest/index.html)

## Other Links
* [List of Links](https://github.com/phillipthelen/awesome-build123d)
* [Parts Library](https://github.com/gumyr/bd_warehouse)
* [Gridfiniy](https://github.com/Ruudjhuu/gridfinity_build123d)
* [Alternate Viewer](https://github.com/yeicor-3d/yet-another-cad-viewer)



In [ ]:
from build123d import *
from ocp_vscode import show, show_object, show_clear
from pathlib import Path
from operations import OperationFace, stock_make
from tools import Tool


In [ ]:

# Alsways need 3 pieces of geometry for a machining operation:
# 1) The part
# 2) The stock
# 3) The tool

# 1) Load a 3D model from a file
fn_part = 'simple-part.step'
part = import_step(fn_part)
part.label = Path(fn_part).stem
part.color = Color("Orange")

# 2) Generate fitting stock
stock = stock_make(part, margin=2)

# 3) Create tool and home it
tool = Tool(diameter=3.175, length=25.4)
tool.to_stock_home(stock)

In [ ]:
f = OperationFace(part=part, tool=tool, stock=stock)
f.woc = tool.diameter*0.4
f.doc = 0.75
f.generate()
toolpath = f.toolpath
f.save_gcode()
stock2 = f.cut(animate=True)


In [ ]:
# Update display
parts = [part, stock, tool, toolpath, stock2]
show(*parts,
     # measure_tools=True,
     #  glass=True,
     )



In [ ]:
tool.material

In [ ]:
# Machined volume
# Machining requires a face or face projection.
# Need to deine a plane of motion for each G0/G1 operation.
# Project part onto plane & do a stock.sweep with that 2D geo.

# removed = sweep()

# Move tool to first position
z = -0.75
vec = Vector(-0.318, -1.75,z)
tool.location = Location(vec)

# First move
edge = Edge.make_line((-0.3175, -0.9525, z), (-0.3175, 30.74625, z))
normal = edge.vertices()[1] - edge.vertices()[0]
normal = Vector(normal)

# Create plane at tool center
origin = tool.position
p = Plane(origin=origin, x_dir=(1, 0, 0), z_dir=-normal)

# Project tool face onto plane
# Projection issue:
# https://github.com/gumyr/build123d/issues/563
# toolface = tool.faces()[0]
# toolface = toolface.project(p)

r = Rectangle(tool.diameter, tool.length)

r = r.translate((0, tool.length / 2))

# To top of stock
z = stock.bounding_box().max.Z
r = r.translate((0, z))


# Cut position
r = r.translate((0, -f.doc))

projection = p * r

cut = sweep(projection, path=edge)
stock2 = stock2 - cut

# Until they've figured out how to project non-planar faces,
# we'll have to use a workaround.
# Create 2D geometry for a rectangle the would be projected.

# Example generic sweep command
# https://build123d.readthedocs.io/en/latest/examples_1.html#peg-board-hook
# mainp = sweep(Plane.YZ * Circle(radius=maind / 2), path=wire)

In [ ]:
# Update display
parts = [part, stock, tool, toolpath, stock2, cut]
show(
    *parts,
    # measure_tools=True,
    #  glass=True,
)